<table align="left" style="border-style: hidden" class="table"> <tr> <td class="col-md-2"><img style="float" src="http://prob140.org/assets/icon256.png" alt="Prob140 Logo" style="width: 120px;"/></td><td><div align="left"><h3 style="margin-top: 0;">Probability for Data Science</h3><h4 style="margin-top: 20px;">UC Berkeley, Spring 2018</h4><p>Ani Adhikari</div></td></tr></table><!-- not in pdf -->

In [ ]:
from prob140 import *
from datascience import *
import numpy as np
import math

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.style.use('fivethirtyeight')

# Lab 5: Uniform and Size Biased Permutations #

Markov chains on permutations of sets of objects have applications in numerous areas, from cryptography to [apple picking](https://link.springer.com/content/pdf/10.1007/978-3-642-19592-1_1.pdf). This lab is an introduction to such chains. You will do some decryption in the next lab, but we will leave you to figure out for yourself how best to pick apples.

Card shuffling has many connections with mathematics, most of all with probability theory. In this lab you will study a simple shuffle known as the *random-to-front*, and see what happens to the deck when you repeat the shuffle many times.

Not all shuffles are based on uniform random sampling. The *Tsetlin Library* is a model for dynamically permuting objects that are randomly selected according to their importance. For example, the objects may be files in a computer system, with some files being accessed more often than others. The long run state of such a library has elegant properties, some of which you will discover and prove.

#### Terminology ####
- An *alphabet* is the finite set of objects that will be permuted. Alphabets in this lab will consist of letters in the usual English alphabet. But we will use only a few letters, not 26.
- Throughout, we will imagine the objects to be set out in a sequence from left to right. The *front* of the permutation is the left-most object.
- *At random* without further qualification means *uniformly at random*. We will start out with uniform random permutations and then move on to a setting where the selection is not uniform.

## Part 1: Permutations in Python ##
As you know, the number of permutations increases rapidly as a function of the number of objects being permuted. Python has functions that help us create all possible permutations of an alphabet.

Start out by running the cell below.

In [ ]:
from itertools import permutations

In this part of the lab we will use just a three-letter alphabet to demonstrate how the functions work. In later parts you can change the alphabet and use the same steps to generate all its permutations.

### 1a) State Space ###
Run the two cells below. The final line evaluates to a list of all possible states.

In [ ]:
alpha_3 = ['A', 'B', 'C']

In [ ]:
states_3 = list(permutations(alpha_3))
states_3

The rest of this part of the lab takes you through `MarkovChain` methods in the `prob140` library. Pay attention to the code as you will need to use it in subsequent parts of the lab.

### 1b) Transition Matrix ###
The transition matrix of the chain depends on the rules by which the chain moves. 
As our first example, we will use the following simple rules for one-step transitions.

- $P(i, i) = 0.5$ for each state $i$
- $P(i, j) = 0.1$ for each pair of states with $j \ne i$

Define a function `transition_rule` to reflect these probabilities. It should take two arguments: $i$ and $j$. It should return $P(i, j)$. 

In [ ]:
def transition_rule(start_state, end_state):
    ...

The cell below shows how to access the elements at index 0 and index 4 of the list `states_3`. 

In [ ]:
abc = states_3[0]
cab = states_3[4]

What should be the output of each of the two following cells? Check that your function returns the right values.

In [ ]:
transition_rule(abc, abc)

In [ ]:
transition_rule(abc, cab)

### 1c) Transition Matrix ###
The call to construct a `MarkovChain` object from a transition function and display its transition matrix is:

`MarkovChain.from_transition_function(list_of_states, name_of_transition_function)`

Construct the chain `lazy_walk` based on the transition behavior in parts **a** and **b**. Check that the transition matrix correctly shows $P(i, j)$ for every pair of states $i$ and $j$. It is "lazy" because it is just as likely to stay in place as it is to move.

In [ ]:
lazy_walk = ...
lazy_walk

Is the chain irreducible? Is it aperiodic? Explain your answers briefly.


**Your answer here.**

### 1d) $n$ Steps ###
For a Markov chain $X_0, X_1, X_2, \ldots$ represented by the `MarkovChain` `mc`, the calls below result in the $n$-step transition matrix and the unconditional distribution of $X_n$.

- `mc.transtion_matrix(n)`
- `mc.distribution(initial_state, n)` where `initial_state` is a given value of $X_0$
- `mc.distribution(initial_distribution, n)` where `initial_distribution` is a `prob140` distribution object containing the distribution of $X_0$ 

Display the 3-step transition matrix of the chain.

In [ ]:
lazy_walk.transition_matrix(3)

Display the distribution of $X_3$ given $X_0 =$ `abc`. Note that the output is a Table; you can use Table methods on it as needed.

In [ ]:
...

### 1e) Steady State ###
What should the steady state distribution of this chain be? Explain your answer.


**Your answer here.**

The call `mc.steady_state()` displays the steady state distribution of the chain. Complete the cell below to create `pi`, the steady state distribution of `lazy_walk`. Check that the result agrees with your answer above.

In [ ]:
pi = ...
pi

If the distribution of $X_0$ is `pi`, what should the distribution of $X_{3}$ be? Explain your answer.


**Your answer here.**

Display the distribution of $X_3$ given that the distribution of $X_0$ is `pi`, and check that it agrees with your answer above.

In [ ]:
...

## Part 2: Random-to-Front Shuffle ##
One of the simplest ways to shuffle a deck of cards is to pick a card at random from the deck and place it at the top, and then repeat the process. This is called the *random-to-top* or *random-to-front* shuffle. In this part you will study this shuffle as a Markov chain.

### 2a) Markov Property ###
Explain why the process of random-to-front shuffling is a Markov chain with time-homogenous transition probabilities. 


**Your answer here.**

### 2b) Move to Front ###
The fundamental move in this process is constructing a permutation of the deck by moving one element to the front, that is, to the extreme left position in the deck. We are helping you do that, by defining the function `move_to_front`. It takes two arguments: a state and the element to move to the front. It returns the state that has the given element moved to the front of the deck and all the others left unchanged.

In [ ]:
def move_to_front(state, element):
    y = list(state)[:]
    y.remove(element)
    return tuple([element] + y) 

Continue with your little deck of three cards for now; we will soon increase the size. If you start with the state `cab` and move `a` to the front, what permutation should you get? Check that the following cells have the right answers. Remember that `abc` is the name to which we assigned `('A', 'B', 'C')` and `cab` is `('C', 'A', 'B')`.

In [ ]:
move_to_front(cab, 'A')

In [ ]:
move_to_front(abc, 'B')

### 2c) Random-to-Front Chain ###
Construct the transition matrix of the random-to-front chain on permutations of `A`, `B`, and `C`. Before you write any code, answer the following question:

Consider the row indexed by the state `('B', 'A', 'C')`. Which elements of the row will be 0? Which will be positive, and what will be their values?

**Your answer here.**

Now complete the cell below so that the last line displays the transition matrix of `random_to_front_3` which is the `MarkovChain` object corresponding to the chain described above. Confirm that it agrees with your answer above.

In [ ]:
def random_to_front_3(start_state, end_state):
    ...
    
random_to_front_3 = MarkovChain ...
random_to_front_3

Explain why the chain is irreducible and aperiodic.


**Your answer here.**

### 2d) Steady State ###
Start with the perfectly ordered state `('A', 'B', 'C')`. We named this `abc` earlier. Find the distribution of the chain after 5 shuffles.

In [ ]:
...

What should the steady state of the chain be? What should the steady state be if the deck has $N$ cards instead of 3? Why?


**Your answer here.**

### 2e) Larger Deck ###
Complete the cell below to check your answer to part **d** for a deck of 6 cards.

In [ ]:
alpha_6 = ['A', 'B', 'C', 'D', 'E', 'F']
states_6 = ...

def random_to_front_6_trans(start_state, end_state):
    ...
           
random_to_front_6 = MarkovChain...
random_to_front_6.steady_state()

Explain by calculation where the value at index 0 of the column `Probability` comes from.

In [ ]:
...

## Part 3: The Tsetlin Library ##
The [Tsetlin Library](https://www.math.ucdavis.edu/~anne/SQ2014/thematic_tutorials/algebraic_combinatorics/tsetlin_library.html) imagines that books are being permuted according to a move-to-front scheme in which the books are chosen not uniformly but based on their *importance* or *popularity*. More popular books are therefore more likely to be selected and moved to the front of the row, that is, in the leftmost position. [Applications](https://arxiv.org/pdf/0805.0083.pdf) of this model to computer science include dynamic file management and cache management.

### 3a) Weights ###
Consider a library with just three books `'A'`, `'B'`, and `'C'`. Let the "importance" of each book be reflected in the probability with which it is selected to be moved to the left: $p_A = 0.6$, $p_B = 0.3$, $p_C = 0.1$.

In [ ]:
weights_3 = Table().with_column(
    'Book', alpha_3,
    'Weight', [0.6, 0.3, 0.1]
)
weights_3

Recall that a row of a Table is a "row object". The call `tbl.row(index)` evaluates to the row of `tbl` that has the given index.

In [ ]:
weights_3.row(0)

You can access elements of a row just as you can access elements of a list.

In [ ]:
weights_3.row(0)[1]

### 3b) The Markov Chain ###
Construct a `MarkovChain` named `tsetlin_3` that is the Tsetlin library based on the books and weights in `weights`. 

Before you complete the cell, get some paper and work out what the transition matrix should be.

In [ ]:
def tsetlin_3_trans(start_state, end_state):
    ...
           
tsetlin_3 = MarkovChain...
tsetlin_3

### 3c) The Short Run ###
Find the distribution of the chain at time 3, given that it started in `abc`.

In [ ]:
tsetlin_3.distribution...

This distribution is not very easy to understand. But when you run the chain for longer, something very interesting happens.

## Part 4: The Tsetlin Library in Steady State ##
The steady state distribution of the Tsetlin Library chain has beautiful properties.

### 4a) The Leftmost Book ###
Display a distribution `tsetlin_3_pi` that is the steady state distribution of the `tsetlin_3` chain. Notice that it is quite close to the distribution of the chain at time at time 5, which you found in **3c**. With such a small library, you don't need many steps before the library gets close to its steady state.

In [ ]:
tsetlin_3_pi = ...
tsetlin_3_pi

Use the table to find the expected long run proportion of time each of the following events happens. In the cell below your calculations, explain why the answers make sense.

(i) the leftmost book is `'A'`

(ii) the leftmost book is `'B'`

(iii) the leftmost book is `'C'`

In [ ]:
leftmost_A = ...
leftmost_B = ...
leftmost_C = ...

leftmost_A, leftmost_B, leftmost_C


**Your answer here.**

### 4b) Second From Left ###
Use the steady state distribution `tsetlin_3_pi` and `weights` to find the two ratios below:

(i) the expected long run proportion of times that `'A'` is leftmost and `'B'` is second leftmost, relative to the expected long run proportion of times `'A'` is leftmost and `'C'` is second leftmost

(ii) weight of `'B'` relative to weight of `'C'`

In [ ]:
ab_ac = ...
b_c = ...

ab_ac, b_c

What do you notice about the answers?


**Your answer here.**

You should now be able to use `weights` to find the following ratios. So you shouldn't need a code cell.

(i) the expected long run proportion of times `'B'` is leftmost and `'A'` is second leftmost, relative to the expected long run proportion of times `'B'` is leftmost and `'C'` is second leftmost

(ii) the expected long run proportion of times `'C'` is leftmost and `'A'` is second leftmost, relative to the expected long run proportion of times `'C'` is leftmost and `'B'` is second leftmost


**Your answer here.**

(i)

(ii)

### 4c) [on paper] Balance Equations ###

Consider the Tsetlin Library that has books `'A'`, `'B'`, `'C'` with positive weights $p_A$, $p_B$, and $p_C$ respectively. Assume $p_A + p_B + p_C = 1$.

Let $\pi$ be the steady state distribution of the chain. 

Write all the balance equations.


### 4d) [on paper] Proportion of Time Spent Leftmost ###
Use the balance equations to find the expected long run proportion of times when `'A'` is leftmost. Your answer should be in terms of the weights.

### 4e) [on paper] Relative Proportion of Time Spent Second from Left ###
Use the balance equations to find the ratio of the following two quantities. Your answer should be in terms of the weights.

- expected long run proportion of times when `'A'` is leftmost and `'B'` is second from left 
- expected long run proportion of times when `'A'` is leftmost and `'C'` is second from left 

## Part 5: A Larger Library ##
These properties are true for larger libraries and any positive weights that add up to 1. Just label the highest weighted book `'A'`, the next highest one `'B'`, and so on, for consistency of interpretation.


### 5a) Steady State ###
Complete the cells below to construct the Tsetlin Library chain with the six books `alpha_6` and any set of positive weights that you make up, as long as $p_A$ is the largest, $p_B$ the second largest, and so on, and the sum of the weights is 1. Find the steady state distribution `tsetlin_6_pi`. It takes a while to do this, so be patient as the cell runs.

In [ ]:
weights_6 = Table().with_column(
    'Book', alpha_6,
    'Weight', ...
)
weights_6

In [ ]:
def tsetlin_6_trans(start_state, end_state):
    ...
           
tsetlin_6 = MarkovChain...
tsetlin_6_pi = tsetlin_6.steady_state()

In [ ]:
tsetlin_6_pi

### 5b) Long Run Proportions ###
(i) Use `tsetlin_6_pi` to find the expected long run proportion of times when `'A'` is leftmost.

It might help to remember that `tbl.take(range_of_indices)` evaluates to a Table consisting of the rows of `tbl` indexed by elements of `range_of_indices`.

In [ ]:
# (i)


(ii) Use `tsetlin_6_pi` to find the ratio of the expected long run proportions of times when:

- `'A'` is leftmost and `'B'` is second from left
- `'A'` is leftmost and `'C'` is second from left

In [ ]:
# (ii)
numerator = ...
denominator = ...

numerator / denominator

(iii) Use `weights_6` to find the ratio of the weight of `'B'` and the weight of `'C'`.

## Conclusion ##
In this lab, you have learned:
- How to work with the `MarkovChain` class in the `prob140` library
- Repeating the random-to-front shuffle leads to a well shuffled deck
- Books in the Tsetlin Library are found in the leftmost position according to their "importance" as measured by their assigned weights
- Among the times when a particular book is on the left, the proportions of times the other books are second from left have the same ratios as the weights of those books
- Markov chains on permutations (both random and size-biased as in the Tsetlin Library) have symmetries that can lead to remarkable long run properties.

## Checklist

Your lab submission will consist of two parts: a written section and a code section

**Written**
- 4c, 4d, 4e

**Code**:

- 1a-e, 2a-e, 3a-c, 4a-b, 5a-b

In [ ]:
import gsExport
gsExport.generateSubmission("Lab_05.ipynb")